In [ ]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
#from keras.models import load_model
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
#from data_split import data_split
from many_net import   many_net_hd_u1d, many_net_1d, many_net_3d_u2d, many_net_uhd

Importing the data

In [ ]:
import csv
trajs_from_files = csv.reader(open('challenge_for_scoring/task1.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

In [ ]:
len(validation[1])

Importing the networks

In [ ]:
centers1d =[25, 50, 65, 75, 125, 165, 225,
            325, 425, 525, 625, 725, 825, 925]
meta_model1d=[]
for i in centers1d: #NB I should go the whole way to 925 and use 25 and 60
    m=load_model('nets/task1/1d/Model_1D_recdout_'+str(i)+'.h5')
    #m=tf.keras.models.load_model('Model_1D_recdout_'+str(i)+'.h5')


    meta_model1d.append(m)
    

In [ ]:
predictions1d=many_net_1d(meta_model1d,validation[0],centers=centers1d)

In [ ]:
np.save('predictions/task1/task1_pred_1d_final.npy',predictions1d)

# 2d case

Using 1d nets

In [ ]:
a, predictions2d_u1d = many_net_hd_u1d(meta_model1d, validation[1],
                                     centers=centers1d,dim=2,thr=1e-12)

In [ ]:
np.save('predictions/task1/task1_pred_2d_u1d_final.npy',predictions2d_u1d)

In [ ]:
predictions2d_u1d.shape

Using 2d nets

In [ ]:
net_centers2d=[25, 65, 125, 225, 325,  525, 725]
meta_model2d=[]
for i in net_centers2d: 
    m=load_model('nets/task1/2d/task1_2D_checkpoint_'+str(i)+'.h5')
    #m=tf.keras.models.load_model('Model_1D_recdout_'+str(i)+'.h5')


    meta_model2d.append(m)
    

In [ ]:
len(validation[1])

In [ ]:
predictions2d_u2d = many_net_uhd(meta_model2d,validation[1],
                                 centers=net_centers2d,dim=2)

In [ ]:
np.save('predictions/task1/task1_pred_2d_u2d_final.npy',predictions2d_u2d)

Classifying which trajectories are LW

In [ ]:
net_centers2d_classi = [25,125, 225,425]
meta_model2d_classi = []
for i in net_centers2d_classi: #NB I should go the whole way to 925 and use 25 and 60
    m=load_model('nets/task2/2d/task2_2D_checkpoint_'+str(i)+'.h5')
    #m=tf.keras.models.load_model('Model_1D_recdout_'+str(i)+'.h5')


    meta_model2d_classi.append(m)
    

In [ ]:
classi_2d = many_net_uhd(meta_model2d_classi, validation[1], 
                         centers=net_centers2d_classi,dim=2)


In [ ]:
classi_2d=classi_2d.reshape(-1,5)

In [ ]:
classi2d_num=np.argmax(classi_2d,axis=1)

In [ ]:
np.save('predictions/task1/task1_classi2d.npy',classi2d_num)

In [ ]:
aa=classi2_num!=3

# 3d case

In [ ]:
a3, predictions3d_u1d = many_net_hd_u1d(meta_model1d,validation[2], 
                                        centers=centers1d, dim=3,
                                        thr=1e-12)

In [ ]:
np.save('predictions/task1/task1_pred_3d_u1d_final.npy',predictions3d_u1d)

3d using 2d nets

In [ ]:
predictions3d_u2d = many_net_3d_u2d(meta_model2d,validation[2],
                                    centers=net_centers2d)

In [ ]:
np.save('predictions/task1/task1_predictions3d_u2d_final.npy',predictions3d_u2d)

Classify the trajectories using nets trained on 2d data on the 2d projections of the 3d data

In [ ]:
classi3d_u2d = many_net_3d_u2d(meta_model2d_classi,validation[2],
                               centers=net_centers2d_classi)

In [ ]:
classi3d_u2d = classi3d_u2d.reshape(-1,5)
num_classi3d_u2d = np.argmax(classi3d_u2d,axis=1)

In [ ]:
np.save('predictions/task1/task1_classi3d_u2d.npy',num_classi3d_u2d)